![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [13]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Import any additional modules and start coding below
from sklearn.ensemble import RandomForestRegressor

In [14]:
df = pd.read_csv('rental_info.csv')
df.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


In [15]:
# Creating `rental_length_days`
# Converting dates to datetime and compute rental duration.
df['rental_date'] = pd.to_datetime(df['rental_date'])
df['return_date'] = pd.to_datetime(df['return_date'])

df['rental_length_days'] = (df['return_date'] - df['rental_date']).dt.days

print(df['rental_length_days'])

0        3
1        2
2        7
3        2
4        4
        ..
15856    6
15857    4
15858    9
15859    8
15860    6
Name: rental_length_days, Length: 15861, dtype: int64


In [16]:
# Dummy Variables from special_features

df['deleted_scenes'] = df['special_features'].apply(
    lambda x: 1 if 'Deleted Scenes' in x else 0
)

df['behind_the_scenes'] = df['special_features'].apply(
    lambda x: 1 if 'Behind the Scenes' in x else 0
)

#### Creating feature matrix X
To avoid data leakage by removing:
- rental_date
- return_date
- rental_length_days (target)

In [17]:
X = df.drop(
    columns=[
        'rental_length_days',
        'rental_date',
        'return_date',
        'special_features'
    ]
)

In [18]:
# Target variable `y`
y = df['rental_length_days']

In [19]:
# Train-test split (20% test set)
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size = 0.2,
    random_state=9
)

In [20]:
# Train a regression model

best_model = RandomForestRegressor(
    n_estimators=200,
    random_state=9
)

best_model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=200, random_state=9)

In [21]:
# Evaluate model perfomance
y_pred = best_model.predict(X_test)
best_mse = mean_squared_error(y_test, y_pred)

best_rmse = best_mse**(1/2)

# Print the test set to MSE and RMSE
print('Test set MSE of best_model: {:.2f}'.format(best_mse))
print('Test set RMSE of best_model: {:.2f}'.format(best_rmse))

Test set MSE of best_model: 2.03
Test set RMSE of best_model: 1.42
